In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.io import imread
import cv2

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from glob import glob
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
!mkdir ~/.keras
!mkdir ~/.keras/models
!cp ../input/keraspretrainedmodels/keras-pretrained-models/*notop* ~/.keras/models/
!cp ../input/keraspretrainedmodels/keras-pretrained-models/imagenet_class_index.json ~/.keras/models/
#!cp ../input/keraspretrainedmodels/keras-pretrained-models/resnet50* ~/.keras/models/

In [ ]:
# ../input/
PATH = os.path.abspath(os.path.join('..', 'input'))

# ../input/sample/images/
SOURCE_IMAGES = os.path.join(PATH, "sample", "sample","images")

# ../input/sample/images/*.png
images = glob(os.path.join(SOURCE_IMAGES, "*.png"))

# Load labels
labels = pd.read_csv('../input/sample/sample_labels.csv')

In [ ]:
# load in all of the images
labels['path'] = labels['Image Index'].apply(lambda x: os.path.join(SOURCE_IMAGES,x))
#labels['image'] = labels['path'].map(imread)

In [ ]:
# see the image size distribution
#labels['image'].map(lambda x: x.shape).value_counts()

In [ ]:
labels['NumLabel']=labels['Finding Labels'].apply(lambda x: 0 if 'No Finding' in x else 1)

In [ ]:
labels.groupby('NumLabel').count()

In [ ]:
labels.head(5)

labels['NumLabel']=labels['Finding Labels'].apply(lambda x: 'No Finding' if 'No Finding' in x else 'Found!')

from itertools import chain
all_labels = np.unique(list(chain(labels['NumLabel'])))
all_labels = [x for x in all_labels if len(x)>0]
print('All Labels ({}): {}'.format(len(all_labels), all_labels))
for c_label in all_labels:
    if len(c_label)>1: # leave out empty labels
        labels[c_label] = labels['NumLabel'].map(lambda finding: 1.0 if c_label in finding else 0)
labels.sample(13)

# keep at least 1000 cases
MIN_CASES = 500
all_labels = [c_label for c_label in all_labels if labels[c_label].sum()>MIN_CASES]
print('Clean Labels ({})'.format(len(all_labels)), 
      [(c_label,int(labels[c_label].sum())) for c_label in all_labels])

labels['disease_vec'] = labels.apply(lambda x: [x[all_labels].values], 1).map(lambda x: x[0])

labels.head(13)

In [ ]:
from sklearn.model_selection import train_test_split 
train_df, test_df = train_test_split(labels, 
                                   test_size = 0.2, 
                                   random_state = 2018,
                                   stratify = labels[['NumLabel', 'Patient Gender']])
print('train', train_df.shape[0], 'validation', test_df.shape[0])

In [ ]:
#from keras.applications.xception import Xception, preprocess_input
#from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.vgg16 import VGG16, preprocess_input
#from keras.applications.inception_v3 import InceptionV3, preprocess_input
#from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, RMSprop
from keras import regularizers

In [ ]:
IMG_SIZE = (128, 128)
core_idg = ImageDataGenerator(preprocessing_function=preprocess_input,
                              samplewise_center=True, 
                              samplewise_std_normalization=True, 
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              rescale = 1.0/255,
                              height_shift_range= 0.15, 
                              width_shift_range=0.15, 
                              rotation_range=40, 
                              shear_range = 0.15,
                              fill_mode = 'nearest',
                              zoom_range=0.15)
val_idg = ImageDataGenerator(preprocessing_function=preprocess_input,
                            rescale = 1.0/255)

In [ ]:
def flow_from_dataframe(img_data_gen, in_df, path_col, y_col, **dflow_args):
    base_dir = os.path.dirname(in_df[path_col].values[0])
    print('## Ignore next message from keras, values are replaced anyways')
    df_gen = img_data_gen.flow_from_directory(base_dir, 
                                     class_mode = 'sparse',
                                    **dflow_args)
    df_gen.filenames = in_df[path_col].values
    df_gen.classes = np.stack(in_df[y_col].values)
    df_gen.samples = in_df.shape[0]
    df_gen.n = in_df.shape[0]
    df_gen._set_index_array()
    df_gen.directory = '' # since we have the full path
    print('Reinserting dataframe: {} images'.format(in_df.shape[0]))
    return df_gen

In [ ]:
train_gen = flow_from_dataframe(core_idg, train_df, 
                             path_col = 'path',
                            y_col = 'NumLabel', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = 32)

valid_gen = flow_from_dataframe(val_idg, test_df, 
                             path_col = 'path',
                            y_col = 'NumLabel', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = 32) # we can use much larger batches for evaluation
# used a fixed dataset for evaluating the algorithm
test_X, test_Y = next(flow_from_dataframe(val_idg, 
                               test_df, 
                             path_col = 'path',
                            y_col = 'NumLabel', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = 32)) # one big batch

In [ ]:
print(test_Y.shape)

t_x, t_y = next(train_gen)
fig, m_axs = plt.subplots(4, 4, figsize = (16, 16))
for (c_x, c_y, c_ax) in zip(t_x, t_y, m_axs.flatten()):
    c_ax.imshow(c_x[:,:,0], cmap = 'bone', vmin = -1.5, vmax = 1.5)
    c_ax.set_title(', '.join([n_class for n_class, n_score in zip(all_labels, c_y) 
                             if n_score>0.5]))
    c_ax.axis('off')

def get_data():
    imgs_x = []
    label_y = []
    for index, row in labels.iterrows():
        if row['image'].shape == (1024,1024):
            imgs_x.append(cv2.resize(row['image'], (150,150), interpolation=cv2.INTER_AREA))
            label_y.append(row['NumLabel'])
    return imgs_x, label_y    

X, Y = get_data()
X = np.array(X)/255
X= np.reshape(X, X.shape + (1,))
Y = np.array(Y)

X.shape

from sklearn.model_selection import train_test_split    
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.18, random_state=2018)
print('Train Shape', X_train.shape, 'test shape', X_test.shape)

# from keras.models import Sequential
from keras.models import Sequential
#from keras.applications.mobilenet import MobileNet
from keras.utils.vis_utils import plot_model
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.optimizers import Adam, SGD, RMSprop
from keras import regularizers
from keras.callbacks import EarlyStopping

In [ ]:
base_model = VGG16(input_shape =  (IMG_SIZE[0],IMG_SIZE[1],3), 
                                 include_top = False, weights = 'imagenet')
#base_model.trainable = False
for layer in base_model.layers[:-4]:
    layer.trainable= False
for layer in base_model.layers:
    if 'bn' in layer.name:
        layer.trainable = False
    print(layer,layer.trainable)
print (len(base_model.layers))
multi_disease_model = Sequential()
multi_disease_model.add(base_model)
multi_disease_model.add(Dropout(0.25))
multi_disease_model.add(GlobalAveragePooling2D())
multi_disease_model.add(Dense(512, activation = 'relu'))
#multi_disease_model.add(Dense(512,activation = 'relu',kernel_regularizer=regularizers.l2(0.0001)))
multi_disease_model.add(BatchNormalization())
#multi_disease_model.add(Dropout(0.5))
multi_disease_model.add(Dense(1, activation = 'sigmoid'))
multi_disease_model.compile(optimizer = "adam", loss = 'binary_crossentropy',
                           metrics = ['accuracy', 'mae'])
multi_disease_model.summary()


In [ ]:
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Input, Conv2D, multiply, LocallyConnected2D, Lambda, AvgPool2D
from keras.models import Model
from keras.layers import BatchNormalization

base_model = VGG16(input_shape =  (IMG_SIZE[0],IMG_SIZE[1],3), 
                                 include_top = False, weights = 'imagenet')
base_model.trainable = False
#for layer in base_model.layers[:-4]:
#    layer.trainable= False
for layer in base_model.layers:
    if 'bn' in layer.name:
        layer.trainable = False
    print(layer,layer.trainable)

pt_features = Input(base_model.get_output_shape_at(0)[1:], name = 'feature_input')
pt_depth = base_model.get_output_shape_at(0)[-1]
#print(base_model.get_output_shape_at(0)[-1])
bn_features = BatchNormalization()(pt_features)

# here we do an attention mechanism to turn pixels in the GAP on an off
attn_layer = Conv2D(128, kernel_size = (1,1), padding = 'same', activation = 'elu')(bn_features)
attn_layer = Conv2D(32, kernel_size = (1,1), padding = 'same', activation = 'elu')(attn_layer)
attn_layer = Conv2D(16, kernel_size = (1,1), padding = 'same', activation = 'elu')(attn_layer)
attn_layer = AvgPool2D((2,2), strides = (1,1), padding = 'same')(attn_layer) # smooth results
attn_layer = Conv2D(1, 
                    kernel_size = (1,1), 
                    padding = 'valid', 
                    activation = 'sigmoid')(attn_layer)
# fan it out to all of the channels
up_c2_w = np.ones((1, 1, 1, pt_depth))
up_c2 = Conv2D(pt_depth, kernel_size = (1,1), padding = 'same', 
               activation = 'linear', use_bias = False, weights = [up_c2_w])
up_c2.trainable = False
attn_layer = up_c2(attn_layer)

mask_features = multiply([attn_layer, bn_features])
gap_features = GlobalAveragePooling2D()(mask_features)
gap_mask = GlobalAveragePooling2D()(attn_layer)
# to account for missing values from the attention model
gap = Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features, gap_mask])
gap_dr = Dropout(0.5)(gap)
dr_steps = Dropout(0.5)(Dense(128, activation = 'elu')(gap_dr))
out_layer = Dense(1, activation = 'sigmoid')(dr_steps)

attn_model = Model(inputs = [pt_features], outputs = [out_layer], name = 'attention_model')

attn_model.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                           metrics = ['binary_accuracy'])

attn_model.summary()    

In [ ]:
from keras.models import Sequential
from keras.optimizers import Adam
tb_model = Sequential(name = 'combined_model')
base_model.trainable = False
tb_model.add(base_model)
tb_model.add(attn_model)
tb_model.compile(optimizer = Adam(lr = 1e-3), loss = 'binary_crossentropy',
                           metrics = ['binary_accuracy'])
tb_model.summary()

In [ ]:

base_mobilenet_model = MobileNet(input_shape =  t_x.shape[1:], 
                                 include_top = False, weights = None)

model = Sequential()

# conv1
model.add(Conv2D(16, kernel_size=(3,3),activation='relu', padding='same',input_shape=(128,128,3),kernel_regularizer=regularizers.l2(0.0001)))
model.add(Conv2D(16, kernel_size=(3,3),activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# conv2
#model.add(base_mobilenet_model)
#model.add(GlobalAveragePooling2D())
#model.add(Dropout(0.5))
model.add(Conv2D(32, kernel_size=(3,3),activation='relu', padding='same',kernel_regularizer=regularizers.l2(0.0001)))
model.add(Conv2D(32, kernel_size=(3,3),activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

#conv3
model.add(Conv2D(64, kernel_size=(3,3),activation='relu', padding='same',kernel_regularizer=regularizers.l2(0.0001)))
model.add(Conv2D(64, kernel_size=(3,3),activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

#conv4
model.add(Conv2D(96, kernel_size=(3,3),dilation_rate = (2,2), activation='relu', padding='same',kernel_regularizer=regularizers.l2(0.0001)))
model.add(Conv2D(96, kernel_size=(3,3),activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

#conv5
model.add(Conv2D(128, kernel_size=(3,3),dilation_rate = (2,2), activation='relu', padding='same',kernel_regularizer=regularizers.l2(0.0001)))
model.add(Conv2D(128, kernel_size=(3,3),activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# fc
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(512, activation='relu',kernel_regularizer=regularizers.l2(0.0001)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy','mae'])
model.summary()


In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

weight_path="{}_weights.best.hdf5".format('xray_class')
checkpoint = ModelCheckpoint(weight_path, monitor='val_acc', verbose=1, 
                             save_best_only=True, mode='max', save_weights_only = True)

early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=5)
callbacks_list = [checkpoint, early]

In [ ]:
multi_disease_model.fit_generator(train_gen, 
                    steps_per_epoch=train_gen.samples//train_gen.batch_size,
                    validation_data = valid_gen,
                    validation_steps = valid_gen.samples//valid_gen.batch_size,
                    epochs = 4, 
                    callbacks = callbacks_list)

history = model.fit(X_train, y_train, epochs = 10, batch_size = 32, verbose=1, validation_data=(X_test, y_test),callbacks=[EarlyStopping(monitor='val_binary_accuracy', patience=5)])

In [ ]:
history = multi_disease_model.fit_generator(train_gen, 
                            steps_per_epoch = train_gen.samples//train_gen.batch_size,
                            validation_data =  valid_gen, 
                            validation_steps = valid_gen.samples//valid_gen.batch_size,
                            epochs = 15, 
                            callbacks = callbacks_list)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#fig.savefig('model_accuracy.png', dpi=300)
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#fig.savefig('model_loss.png', dpi=300)